In [1]:
import numpy as np
import torch
from torch import nn, optim, autograd
from torch.nn import functional as F
from pyDOE import lhs
import scipy.io
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.gridspec as gridspec
%matplotlib inline
from mpl_toolkits.axes_grid1 import make_axes_locatable

from models_all import *

#Paper reproduction
torch.manual_seed(1234)
torch.cuda.manual_seed(1234)
np.random.seed(1234)

In [2]:
N_train = 10000
N_bound = 200

la = np.array([1,1])
lb = np.array([-1,0])

traindata = lb+(la-lb)*lhs(2,N_train)

bound_x = -1+(1-(-1))*lhs(1,N_bound)

bound_t = 0+(1-(0))*lhs(1,N_bound)

# #x,t  #u,v
X_train = torch.from_numpy(traindata).float()
# x_bound = torch.from_numpy(bound_x).float()
# t_bound = torch.from_numpy(bound_t).float()
x = X_train[:,0:1]
t = X_train[:,1:2]

x.requires_grad_()
t.requires_grad_()

###########GPU###########
# x_bound = x_bound.cuda()
# t_bound = t_bound.cuda()
x = x.cuda()
t = t.cuda()
###########GPU###########

In [3]:
import scipy.io
data = scipy.io.loadmat('./burgers_shock.mat')
t_ = data['t'].flatten()[:,None]
x_ = data['x'].flatten()[:,None]
Exact = np.real(data['usol']).T

X, T = np.meshgrid(x_,t_)

X_star = np.hstack((X.flatten()[:,None],T.flatten()[:,None]))
u_star = Exact.flatten()[:,None]  

In [4]:
np.random.seed(1234)
n=200

index_t = (lhs(1,n))*len(t_)
index_t = np.floor(index_t).reshape(n,).astype(int)
t_star = t_[index_t]

index_x = (lhs(1,n))*len(x_)
index_x = np.floor(index_x).reshape(n,).astype(int)
x_star = x_[index_x]

x_t_star = np.hstack((x_star,t_star))
u_star_ = Exact[index_t,index_x]

In [5]:
observe_data = x_t_star
observe_u = u_star_.reshape(-1,1)
observe_data = torch.from_numpy(observe_data).float()
observe_u = torch.from_numpy(observe_u).float()

In [6]:
C1 = torch.tensor(0.5, requires_grad=True)

In [7]:
###########GPU###########
observe_u = observe_u.cuda()
observe_data = observe_data.cuda()
###########GPU###########

In [8]:
np.random.seed(5678)
index_t = (lhs(1,10000))*len(t_)
index_t = index_t.astype(int).reshape(10000,)
t_star = t_[index_t]
index_x = (lhs(1,10000))*len(x_)
index_x = index_x.astype(int).reshape(10000,)
x_star = x_[index_x]
test_data = np.hstack((x_star,t_star))
test_u = Exact[index_t,index_x].reshape(-1,1)

test_data = torch.from_numpy(test_data).float()
test_u = torch.from_numpy(test_u).float()

###########GPU###########
test_data = test_data.cuda()
test_u = test_u.cuda()
###########GPU###########

In [9]:
def relative_l2(u_pred, u_real):

    l2 = torch.norm(u_real - u_pred, p=2) / torch.norm(u_real, p=2)
    
    return l2.item()  # Convert the result back to a Python float

In [10]:
def output_transform(x, y):
    x_in = x[:, 0:1]
    t_in = x[:, 1:2]

    return (1 - x_in) * (1 + x_in) * (1 - torch.exp(-t_in)) * y - torch.sin(np.pi * x_in)

In [11]:
def Loss_f(x_inside,y_inside,PINNs,C,return_sequence='not'):
    
    ########### loss f  ###########
    E_inside = PINNs(torch.cat((x_inside,y_inside),1))
    E_inside = output_transform(torch.cat((x_inside,y_inside),axis=1), E_inside)
    
    
    E_x = autograd.grad(outputs=E_inside, inputs=x_inside,
                              grad_outputs=torch.ones_like(E_inside),
                              create_graph=True, retain_graph=True, only_inputs=True)[0]
    E_xx = autograd.grad(outputs=E_x, inputs=x_inside,
                              grad_outputs=torch.ones_like(E_x),
                              create_graph=True, retain_graph=True, only_inputs=True)[0]
    E_t = autograd.grad(outputs=E_inside, inputs=y_inside,
                              grad_outputs=torch.ones_like(E_inside),
                              create_graph=True, retain_graph=True, only_inputs=True)[0]     
    ########### loss f  ###########
    
    if return_sequence=='yes':
        return torch.square(E_t+E_inside*E_x-C*(E_xx))
    else:
        return torch.mean(torch.square(E_t+E_inside*E_x-C*(E_xx)))

## PINN

In [ ]:
torch.manual_seed(1234)
torch.cuda.manual_seed(1234)
np.random.seed(1234)


PINNs1 = NN_H2(2, 20, 7, 1)
PINNs1.cuda()

#PINNs1.apply(weights_init)
import torch.nn.init as init
for name, param in PINNs1.named_parameters():
    if 'weight' in name:
        init.xavier_uniform_(param)


optimizer1 = optim.Adam(PINNs1.parameters(), lr=0.001,betas=(0.9, 0.999), eps=1e-08, weight_decay=0, amsgrad=False)

optimizer1.add_param_group({'params': [C1], 'lr': 0.001})

loss_all_1 = []
loss_f_1 = []
loss_T_1 = []
test_loss_1 = []
C1_list = []
# 0.01
beta_number_list_A = []
# 0.005
beta_number_list_B = []

traindata_batch_list = []
bound_x_batch_list = []
bound_t_batch_list = []

nIter1 = 10000

loss1_value = 1
it = 0
while it<nIter1:
   
    ##### loss f  ######
    loss_f = Loss_f(x,t,PINNs1,C1)
    
    ##### loss observation  ######        
    E_observation = PINNs1(observe_data) 
    E_observation = output_transform(observe_data, E_observation)    
    loss_observation = torch.mean(torch.square(E_observation-observe_u))         
    ##### loss observation  ######       
    
    #####loss PI#######
    loss = loss_f+10*loss_observation 
    #####loss PI#######
    
    #########  test_loss NRMSE  #########
    pre_u = PINNs1(test_data)
    pre_u = output_transform(test_data, pre_u)      
    test_loss = relative_l2(pre_u,test_u)
    #########  test_loss NRMSE  #########
    
    ##############Record###############
    loss_f_1.append(loss_f.item()) 
    test_loss_1.append(test_loss)
    C1_list.append(C1.item())   
    loss_T_1.append(loss_observation.item()) 
    loss_all_1.append(loss.item())
    ##############Record###############
    
    ############### beta_number ###############
    loss_f_item = Loss_f(x,t,PINNs1,C1,return_sequence='yes').detach().cpu().numpy()
    # 0.01
    beta_number_list_A.append(np.sum(loss_f_item <= 0.01)/loss_f_item.shape[0])
    # 0.005
    beta_number_list_B.append(np.sum(loss_f_item <= 0.005)/loss_f_item.shape[0])
    ############### beta_number ###############
    
    
    optimizer1.zero_grad()
    loss.backward()
    optimizer1.step()
    
    if it % 1000 == 0:
        ###########GPU###########
        print('It:', it, 'train_loss:', loss.item(),'test_loss:', test_loss)
        print('C1',C1)     
    it = it + 1        

print('Final:', 'train_loss:', loss.item(), 'test_loss:', test_loss)